In [ ]:
import subprocess

# List of required libraries
required_libraries = ["docx2txt", "scikit-learn", "tkinter"]

# Check and install required libraries
for library in required_libraries:
    try:
        # Check if the library is already installed
        __import__(library)
    except ImportError:
        # If not installed, install the library
        subprocess.call(["pip", "install", library])

# Now import the libraries
import docx2txt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import filedialog, messagebox

def load_resume_text(resume_filename):
    try:
        resume_text = docx2txt.process(resume_filename)
        return resume_text
    except FileNotFoundError:
        print(f"File not found: {resume_filename}")
        return ""

def calculate_similarity_and_show_message(resume_filename, job_description_text):
    try:
        resume_text = load_resume_text(resume_filename)
        if resume_text:
            # Print the extracted resume text (for debugging purposes)
            print(f"\nExtracted Resume Text ({resume_filename}):")
            print(resume_text)

            # Create a CountVectorizer to convert text to vectors
            cv = CountVectorizer()
            count_matrix = cv.fit_transform([resume_text, job_description_text])

            # Print the count matrix (for debugging purposes)
            print("\nCount Matrix:")
            print(count_matrix.toarray())

            # Calculate cosine similarity between the two documents
            cosine_sim = cosine_similarity(count_matrix)
            match_percentage = round(cosine_sim[0][1] * 100, 2)

            # Determine the color based on the match percentage for the console output
            color = get_color_based_on_percentage(match_percentage)

            # Display the message with colored text in the console output
            print(f"\n\033[{color}mThe resume matches {match_percentage:.2f}% to the job description.\033[0m")

            # Determine the color based on the match percentage for the Tkinter widget
            widget_color = get_widget_color_based_on_percentage(match_percentage)

            # Display the message with colored text in the Tkinter Label widget
            message_label.config(text=f"The resume matches {match_percentage:.2f}% to the job description.", fg=widget_color)

            # Determine the category based on the match percentage
            category = get_resume_category(match_percentage)

            # Print the dynamic accuracy for the current resume
            dynamic_accuracy = round(calculate_dynamic_accuracy(resume_filename, job_description_text), 2)
            print(f"\nDynamic Accuracy ({resume_filename}): {dynamic_accuracy}%")

            # Show a popup message with the match percentage and category
            popup_message = f"The resume matches {match_percentage:.2f}% to the job description.\nCategory: {category}"
            messagebox.showinfo("Resume Matching Result", popup_message)
    except Exception as e:
        handle_error(e)

def calculate_dynamic_accuracy(resume_filename, job_description_text):
    try:
        resume_text = load_resume_text(resume_filename)
        if resume_text:
            dynamic_similarity = calculate_similarity(resume_text, job_description_text)
            return dynamic_similarity
    except Exception as e:
        handle_error(e)
        return 0.0

def handle_error(error):
    # Display the error message in the console output and Tkinter widget
    error_message = f"\n\033[91mAn unexpected error occurred: {error}\033[0m"
    print(error_message)
    message_label.config(text=f"An unexpected error occurred: {error}", fg="red")
    # Show a popup message for the error
    messagebox.showerror("Error", f"An unexpected error occurred: {error}")

def browse_resume():
    try:
        browse_button.config(state=tk.DISABLED)
        resume_filename = filedialog.askopenfilename(title="Select Resume File", filetypes=[("Word Documents", "*.docx")])
        browse_button.config(state=tk.NORMAL)
        if resume_filename:
            job_description = job_description_text.get("1.0", tk.END)
            calculate_similarity_and_show_message(resume_filename, job_description)
    except Exception as e:
        handle_error(e)

def calculate_similarity(resume_text, job_description_text):
    cv = CountVectorizer()
    count_matrix = cv.fit_transform([resume_text, job_description_text])
    cosine_sim = cosine_similarity(count_matrix)
    match_percentage = round(cosine_sim[0][1] * 100, 2)
    return match_percentage

def get_color_based_on_percentage(match_percentage):
    if match_percentage < 50:
        return "91"  # Red
    elif match_percentage < 60:
        return "93"  # Yellow
    else:
        return "92"  # Green

def get_widget_color_based_on_percentage(match_percentage):
    if match_percentage < 50:
        return "red"
    elif match_percentage < 60:
        return "orange"
    else:
        return "green"

def get_resume_category(match_percentage):
    if match_percentage < 50:
        return "Rejected"
    elif match_percentage < 60:
        return "On Hold"
    else:
        return "Qualified (Next Stage)"

# Improved user interface design
window = tk.Tk()
window.title("Resume Screening Application")

# Browse button
browse_button = tk.Button(window, text="Browse Resume", command=browse_resume)
browse_button.pack(pady=10)

# Job description text box
job_description_text = tk.Text(window, height=10, width=50)
job_description_text.pack(pady=10)

# Job description label
job_description_label = tk.Label(window, text="Job Description:")
job_description_label.pack(pady=5)

# Default job description
default_job_description_path = "job-description.docx"
job_description_text.insert(tk.END, docx2txt.process(default_job_description_path))

# Message label
message_label = tk.Label(window, text="", fg="black")
message_label.pack(pady=10)

# Main loop
window.mainloop()